In [1]:
import torch
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR  #
# from torch.utils.data import TensorDataset, \
#     DataLoader  # Own stuff set of data quality checks, tensor shapes might be different dataloader loads the tensor,
import pytorch_lightning as pl  #
from pytorch_lightning.callbacks.early_stopping import \
    EarlyStopping  # early stop when you reach optimum loss, 3 times in a row gradient descent
from pytorch_lightning.callbacks import LearningRateMonitor  # delta (loss / accuracy)
from pytorch_lightning.loggers import MLFlowLogger  # Model tracking
import torch
import torch.nn as nn
from transformers import DistilBertModel
from data_preprocessing import injestDataset, downSample
from data_cleaning import feature_cleaner, cleaningPreprocess
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer, LabelBinarizer
from sklearn.compose import ColumnTransformer

import torchmetrics
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel, DistilBertModel, DistilBertTokenizer, DistilBertForTokenClassification
import numpy as np
# from transformers import DistilBertModel

In [2]:
torch.set_default_dtype(torch.float32)
# torch.set_default_device('mps:0')
# torch.set_default_device('cpu:0')

In [3]:
data = injestDataset()

In [4]:
df = downSample(data,"Recommended IND",1000)

In [5]:
df.dtypes

trueIndex           int64
Review Text        object
Rating              int64
Recommended IND     int64
dtype: object

In [6]:
df = cleaningPreprocess(df, "Review Text")

In [7]:
df = df.rename(columns={"Recommended IND": "targetOne", "Review Text": "feature", "Rating": "targetTwo"})

In [8]:
df

,trueIndex,feature,targetTwo,targetOne
0,12849,ive looking sometime springsummer jacket wear ...,5,1
1,4578,great spring time dress looks good thin top ca...,5,1
2,6169,material shirt soft appealing great fall side ...,4,1
3,2942,dress light twirly fabric airy pretty sadly fi...,4,1
4,2769,bought shirt 16 year old daughter looked adora...,4,1
...,...,...,...,...
995,12936,beautiful dress however looked awful ordered m...,1,0
996,258,fits well shoulders arms zero waist looks like...,3,0
997,19148,love maeve dresses several issues one taste 1 ...,3,0
998,20477,okay top redeeming qualities promise gave 3 st...,3,0


In [9]:
# from transformers import AutoTokenizer, DistilBertForTokenClassification
# import torch

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased")

# inputs = tokenizer(
#     "HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt"
# )

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_token_class_ids = logits.argmax(-1)

# # Note that tokens are classified rather then input words which means that
# # there might be more predicted token classes than words.
# # Multiple token classes might account for the same word
# predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

# labels = predicted_token_class_ids
# loss = model(**inputs, labels=labels).loss

In [10]:
# Here is your dataset
class MyDataset(Dataset):
    def __init__(self, df):
        self.input_data = df
        # self.max_length = 30
        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.tokenize_dataframe()
        self.train_val_test_idx()
        self.data_split = None
        self.split_columns()
        self.targertOne_labelEncoder()
        self.targertTwo_labelEncoder()

    def tokenize_dataframe(self):
        tokenized_texts = []
        input_ids = []
        attention_masks = []

        for text in self.input_data['feature']:
            encoded_inputs = self.tokenizer(
                text,
                add_special_tokens=True,
                # max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            tokenized_texts.append(encoded_inputs)
            input_ids.append(encoded_inputs['input_ids'])
            attention_masks.append(encoded_inputs['attention_mask'])
        
        self.input_data['tokenizedFeature'] = tokenized_texts
        self.input_data['input_ids'] = input_ids
        self.input_data['attention_mask'] = attention_masks


    def split_columns(self):
        self.features = self.input_data["feature"]
        self.targetOne = self.input_data["targetOne"]
        self.targetTwo = self.input_data["targetTwo"]

            
    def train_val_test_idx(self):
        idx = list(range(len(self.input_data)))
        train_idx, val_test_idx = train_test_split(
                                idx,
                                train_size=0.8,
                                stratify=self.input_data["targetOne"],
                                random_state=500)
                                
        test_idx,  val_idx, = train_test_split(
                                val_test_idx,
                                train_size=0.5,
                                random_state=500)

        self.train_idx = train_idx 
        self.test_idx = test_idx
        self.val_idx = val_idx
    
    def targertOne_labelEncoder(self):
        # self.targetOne_label_binarizer = MultiLabelBinarizer()
        self.targetOne_label_binarizer = LabelEncoder()
        transformed =  self.targetOne_label_binarizer.fit_transform(self.input_data["targetOne"].astype("str"))
        self.input_data['targetOne_binirized'] = [subarray[0] for subarray in np.split(transformed,len(transformed))]

    def targertTwo_labelEncoder(self):
        self.targetTwo_label_binarizer = MultiLabelBinarizer()
        transformed =  self.targetTwo_label_binarizer.fit_transform(self.input_data["targetTwo"].astype("str"))
        self.input_data['targetTwo_binirized'] = [subarray[0] for subarray in np.split(transformed,len(transformed))]

        
    def __len__(self):
        if self.data_split is None or self.data_split == 'all':
            # Return the length of the entire dataset
            return len(self.input_data)
        elif self.data_split == 'train':
            # Return the length of the training split
            return len(self.train_idx)
        elif self.data_split == 'val':
            # Return the length of the validation split
            return len(self.val_idx)
        elif self.data_split == 'test':
            # Return the length of the test split
            return len(self.test_idx)
        else:
            raise ValueError("Invalid data_split argument. Use 'train', 'val', 'test', or 'all'.")
        
    ## The whole purpose of this __getitem__ function is to return a dataframe or dataloader compatible data 

    def __getitem__(self, index):
        if self.data_split is not None:
            # Depending on the data_split argument, return the corresponding split
            if self.data_split == 'train':
                index = self.train_idx[index]
            elif self.data_split == 'val':
                index = self.val_idx[index]
            elif self.data_split == 'test':
                index = self.test_idx[index]
            else:
                raise ValueError("Invalid data_split argument. Use 'train', 'val', or 'test'.")

            input_ids = self.input_data['input_ids'][index]
            attention_mask = self.input_data['attention_mask'][index]
            targetOne = torch.tensor(self.input_data['targetOne_binirized'][index],dtype=torch.float32,device='mps:0')
            targetTwo = torch.tensor(self.input_data['targetTwo_binirized'][index],dtype=torch.float32,device='mps:0')

            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'targetOne': targetOne,
                'targetTwo': targetTwo,
            }

        else: 
            return self.input_data.iloc[index]

       # Define properties to access different data splits


    # def set_data_split(self, data_split):
    #     # This method allows you to set the data_split after creating the instance
    #     self.data_split = data_split

    # def reset_data_split(self):
    #     # This method allows you to reset the data_split to None, returning the entire DataFrame
    #     self.data_split = None

    @property
    def train(self):
        self.data_split = 'train'
        return self

    @property
    def val(self):
        self.data_split = 'val'
        return self

    @property
    def test(self):
        self.data_split = 'test'
        return self

    @property
    def all(self):
        self.data_split = None
        return self



In [11]:
# class Embedding(nn.Module):
#    def __init__(self):
#        super(Embedding, self).__init__()
#        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
#        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
#        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
#        self.norm = nn.LayerNorm(d_model)

#    def forward(self, x, seg):
#        seq_len = x.size(1)
#        pos = torch.arange(seq_len, dtype=torch.long)
#        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
#        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
#        return self.norm(embedding)

In [12]:
# x, y, z = dataLoader()

In [13]:
# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, output_dim=4, input_dim=512, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         # lower than epoch higher than iterations
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         # pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.pre_classifier = torch.nn.Linear(512, 512)
#         self.dropout = torch.nn.Dropout(0.3)
#         self.classifier = torch.nn.Linear(512,4)

#     def forward(self, batch):
#         # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
#         # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
#         return [optimizer], [scheduler]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         # TODO this will be changed
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().numpy()

In [14]:
# trainer = pl.Trainer(max_epochs=1,accelerator="mps")
# trainer.fit(model)

In [15]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader

In [16]:
# import pytorch_lightning as pl
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from transformers import DistilBertModel
# from torch.optim.lr_scheduler import OneCycleLR
# from torchmetrics.classification import Accuracy

# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self,trainDataLoader,testDataLoader, valDataLoader, seed, output_dim, input_dim=768, hidden_dim=768, learning_rate=1e-3, max_learning_rate=0.1, total_steps=2):
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.pre_classifier = nn.Linear(input_dim, hidden_dim)
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(hidden_dim, output_dim)

#     def forward(self, batch):
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'], return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     # def configure_optimizers(self):
#     #     optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
#     #     scheduler = OneCycleLR(optimizer, max_lr=self.hparams.max_learning_rate, total_steps=2)
#     #     return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['targetOne'])
#         self.log("train_loss", loss, on_step=True, on_epoch=False)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['targetOne'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['targetOne'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().cpu().numpy()
    
#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader


In [17]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader

In [18]:
# def Load_model():
#     my_dataset = MyDataset(df)
#     batch_size=4
#     train_loader = DataLoader(my_dataset.train, batch_size=batch_size, shuffle=True)
#     val_loader = DataLoader(my_dataset.val, batch_size=batch_size, shuffle=False)
#     test_loader = DataLoader(my_dataset.test, batch_size=batch_size, shuffle=False)

#     model = MultiClassClassifier(train_loader,val_loader,test_loader, 500,2)
#     return model

In [19]:
my_dataset = MyDataset(df)
batch_size=5
train_loader = DataLoader(my_dataset.train, batch_size=batch_size, shuffle=True)

In [20]:
# def forward(self, batch):
#     outputs = self.model(batch['input_ids'], attention_mask=batch['attention_mask'], return_dict=True)
#     pooler = torch.nn.ReLU()(self.linear1(outputs.last_hidden_state))
#     pooler = self.dropout(pooler)
#     output = self.linear2(pooler)
#     return output


In [21]:
import torch.nn as nn
# number of features (len of X cols)
input_dim = 521
# number of hidden layers
hidden_layers = 758
# number of classes (unique of y)
output_dim = 2
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.device = "mps:0"
    self.model = DistilBertModel.from_pretrained("distilbert-base-uncased")
    # self.pre_classifier = nn.Linear(521, 521)
    # self.classifier = nn.Linear(521, output_dim)
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
    self.dropout = nn.Dropout(0.1)
    self.activation = nn.ReLU()
  def forward(self, batch):
    embeddings = self.model(batch['input_ids'].squeeze(1).to(self.device), attention_mask=batch['attention_mask'].to(self.device), output_hidden_states= True, return_dict=False)[0]
    pooler = self.activation(self.linear1(embeddings))
    pooler = self.dropout(pooler)
    output = self.linear2(pooler)
    return output

#  def forward(self, x):
#     x = torch.sigmoid(self.linear1(x))
#     x = self.linear2(x)
#     return x

In [22]:
clf = Network()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

In [23]:
for i, batch in enumerate(train_loader, 0):
    input_ids = batch['input_ids'] # Access input IDs from the batch
    attention_mask = batch['attention_mask']  # Access attention masks from the batch
    targetOne = batch['targetOne']  # Access targetOne from the batch
    targetTwo = batch['targetTwo'] 

In [24]:
input_ids_2 = input_ids.squeeze(dim=1)

In [25]:
input_ids.shape

torch.Size([5, 1, 512])

In [28]:
clf = Network()
device = torch.device("mps:0")
clf.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

epochs = 1
for epoch in range(epochs):
    running_loss = 0.0
    for i, batch in enumerate(train_loader, 0):
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['targetOne']  # Assuming 'targetOne' is your target variable
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = clf({'input_ids': inputs, 'attention_mask': attention_mask})
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Display statistics
    print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / len(train_loader):.5f}')


: 

: 

In [ ]:
# clf = Network()
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

# epochs = 2
# for epoch in range(epochs):
#   running_loss = 0.0
#   for i, batch in enumerate(train_loader, 0):
#     # inputs, labels = data
#     # set optimizer to zero grad to remove previous epoch gradients
#     optimizer.zero_grad()
#     # forward propagation
#     outputs = clf(batch)
#     prediction = torch.softmax(outputs,dim=1)
#     new_labels = torch.argmax(batch['targetOne'],dim=1)
#     loss = criterion(outputs, new_labels)
#     # backward propagation
#     loss.backward()
#     # optimize
#     optimizer.step()
#     running_loss += loss.item()
#   # display statistics
#   print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

In [ ]:
# trainer = pl.Trainer(max_epochs=1,accelerator="mps")
# # model = Load_model()
# # trainer.fit(model)# 

In [ ]:
# import pytorch_lightning as pl
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from transformers import DistilBertModel
# from torch.optim.lr_scheduler import OneCycleLR
# from torchmetrics.classification import Accuracy

# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.pre_classifier = nn.Linear(521, 521)
#         self.dropout = nn.Dropout(0.3)
#         self.classifier = nn.Linear(521, output_dim)

#     def forward(self, batch):
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'], return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = optim.AdamW(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, total_steps=self.hparams.total_steps)
#         return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.cross_entropy(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().cpu().numpy()


In [ ]:
# class MultiClassClassifier(pl.LightningModule):
#     def __init__(self, seed, output_dim, input_dim=4, hidden_dim=5, learning_rate=1e-3, max_learning_rate=0.1, total_steps=100):
#         # lower than epoch higher than iterations
#         super().__init__()
#         self.save_hyperparameters()  # save_hyperparameters
#         pl.seed_everything(seed)
#         self.model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         # self.pre_classifier = torch.nn.Linear(768, 768)
#         # self.dropout = torch.nn.Dropout(0.3)
#         # self.classifier = torch.nn.Linear(768,output_dim)

#     def forward(self, batch):
#         # input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
#         # input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
#         embeddings = self.model(batch['input_ids'], attention_mask=batch['attention_mask'],return_dict=False)[0]
#         pooler = self.pre_classifier(embeddings)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

#     def configure_optimizers(self):
#         optimizer = Adam(self.parameters(), lr=self.hparams.learning_rate)
#         scheduler = OneCycleLR(optimizer, self.hparams.max_learning_rate, self.hparams.total_steps)
#         return [optimizer], [scheduler]

#     def training_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("train_loss", loss, on_step=False, on_epoch=True)
#         return loss

#     def validation_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("val_loss", loss)

#     def test_step(self, batch, batch_idx):
#         logits = self.forward(batch)
#         loss = F.nll_loss(logits, batch['labels'])
#         self.log("test_loss", loss)

#     def predict(self, x):
#         # TODO this will be changed
#         self.eval()
#         logits = self.forward(x)
#         self.train()
#         return torch.argmax(logits, dim=1).detach().numpy()

# #     # def df_to_tensor(self, df, target_col=None, format=np.float32):
# #     #     if target_col is not None:
# #     #         feature_cols = [col for col in df.columns if col != target_col]
# #     #         tensor = TensorDataset(
# #     #             torch.tensor(df[feature_cols].values.astype(format)),
# #     #             torch.tensor(df[target_col].values),
# #     #         )
# #     #     else:
# #     #         tensor = torch.tensor(df.values.astype(format))
# #     #     return tensor

# #     def tensor_to_loader(self, tensor, batch_size, num_workers, shuffle=True):
# #         loader = DataLoader(dataset=tensor, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
# #         return loader

# #     def setup_trainer(self, experiment, run_id, max_epochs):
# #         # mlf_logger = MLFlowLogger(experiment_name=experiment)
# #         # mlf_logger._run_id = run_id
# #         trainer = pl.Trainer(
# #             max_epochs=max_epochs,
# #             callbacks=[EarlyStopping(monitor="val_loss"), LearningRateMonitor(logging_interval="epoch")],
# #             # logger=mlf_logger,
# #         )

# #         return trainer

In [ ]:
# class CustomBertClassifier(nn.Module):
#     def __init__(self, bert_model):
#         super(CustomBertClassifier, self).__init__()
#         self.bert = bert_model
#         self.fc = nn.Linear(bert_model.config.hidden_size, 2)  # Binary classification

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids, attention_mask=attention_mask)
#         pooled_output = outputs['pooler_output']
#         logits = self.fc(pooled_output)
#         return logits

In [ ]:

# # Here is your PyTorch Lightning model
# class LitNetwork(pl.LightningModule):
#     def __init__(self, trainDataLoader, testDataLoader, valDataLoader):
#         super(LitNetwork, self).__init__()
#         self.train_ds = trainDataLoader
#         self.test_ds = testDataLoader
#         self.val_ds = valDataLoader
#         self.linear1 = nn.Linear(48, 25)
#         self.linear2 = nn.Linear(25, 2)
#         # self.label_binarizer = trainDataLoader.

#     def forward(self, x):
#         x = torch.sigmoid(self.linear1(x))
#         x = self.linear2(x)
#         return x

#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.functional.cross_entropy(y_hat, y)
        
#         # Calculate accuracy
#         preds = torch.argmax(y_hat, dim=1)
#         acc = torchmetrics.functional.accuracy(preds,  y)

#         # Logging to TensorBoard by default
#         self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#         self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
#         return loss
    

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=0.02)

#     def train_dataloader(self):
#         return self.train_ds

#     def test_dataloader(self):
#         return self.test_ds  # return your dataloader
        
#     def val_dataloader(self):
#         return self.val_ds  # return your dataloader